In [1]:
import os
import re
from typing import List

import pandas as pd
import numpy as np
from datasets import load_dataset

In [2]:
from kowit24_evaluation import check_interpretation

In [3]:
# configs
PROJECT_PATH = "/home/alex/KoWit-24_public"

In [4]:
data_files = {"test": "dataset.csv", "dev": "dev_dataset.csv"}
dataset = load_dataset("Humor-Research/KoWit-24", data_files=data_files)

In [5]:
dataset = pd.DataFrame().from_dict(dataset["test"])
print(dataset.shape)

(2500, 7)


In [6]:
dataset.head(1)

,annotations,summary,is_wordplay,date,article_url,headline,lead
0,None,Жилищный нацпроект помимо количественных целей...,False,2021-12-16,https://www.kommersant.ru/doc/5130217,Комфортная среда,Городскую среду улучшают с учетом мнения жителей


In [7]:
df_gigachat_lite = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_interpretation_gigachat_lite_predictions_extended.json"
    ),
    orient="index"
)

In [8]:
df_gigachat_lite.sample(1)

,user_prompt,system_prompt,gigachat_lite_preds
1383,"Заголовок новости: Лучше сидя, чем на коленях....",\nПроанализируй заголовок новости в контексте ...,"В заголовке новости ""Лучше сидя, чем на коленя..."


In [9]:
df_gigachat_max = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_interpretation_gigachat_max_predictions_extended.json"
    ),
    orient="index"
)

In [10]:
df_gigachat_max["gigachat_max_pred"].fillna("что-то в вашем вопросе меня смущает.", inplace=True)

In [11]:
df_gigachat_max.sample(1)

,user_prompt,system_prompt,gigachat_max_pred
1115,Заголовок новости: Продавленная кнопка. Cодерж...,\nПроанализируй заголовок новости в контексте ...,в заголовке нет игры слов.


In [12]:
df_yagpt = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_interpretation_yandex_gpt4_predictions_extended.json"
    ),
    orient="index"
)

In [13]:
df_yagpt.sample(1)

,user_prompt,system_prompt,yagpt_pred
1378,Заголовок новости: Мирные уговоры. Cодержание ...,\nПроанализируй заголовок новости в контексте ...,в интернете есть много сайтов с информацией на...


In [14]:
df_mistral = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_interpretation_mistral_nemo_predictions_extended.json"
    ),
    orient="index"
)

def clear_mistral_prediction(text_pred):
    
    text = text_pred[0]["generated_text"][-1]["content"]

    return text

df_mistral["mistral_nemo_preds"] = df_mistral["mistral_nemo_preds"].apply(clear_mistral_prediction)

In [15]:
df_mistral.sample(1)

,user_prompt,system_prompt,mistral_nemo_preds
2005,Заголовок новости: Неуходящая натура. Cодержан...,\nПроанализируй заголовок новости в контексте ...,"В заголовке новости ""Неуходящая натура. Открыв..."


In [16]:
df_gpt4o = pd.read_json(
    os.path.join(
        PROJECT_PATH,
        "Data/predictions/wordplay_interpretation_gpt_4o_predictions_extended.json"
    ),
    orient="index"
)

In [17]:
df_gpt4o_prompt = pd.read_json("/home/alex/KoWit-24_public/Data/processed_data/dataset_wordplay_interpretation_propmts_extended.json", orient="index")
df_gpt4o_prompt["task_id"] = df_gpt4o_prompt.index

In [18]:
df_gpt4o = df_gpt4o.merge(df_gpt4o_prompt, how="left", on="task_id")

In [19]:
def extract_headline_from_user_prompt(text):
    idx = text.find("Cодержание новости:")
    return text[19:idx-2]

In [20]:
def extract_lead_from_user_prompt(text):
    idx = text.find("Cодержание новости:")
    return text[idx+20:]

In [21]:
df_gigachat_lite["headline"] = df_gigachat_lite["user_prompt"].apply(extract_headline_from_user_prompt)
df_gigachat_lite["lead"] = df_gigachat_lite["user_prompt"].apply(extract_lead_from_user_prompt)

df_gigachat_max["headline"] = df_gigachat_max["user_prompt"].apply(extract_headline_from_user_prompt)
df_gigachat_max["lead"] = df_gigachat_max["user_prompt"].apply(extract_lead_from_user_prompt)

df_yagpt["headline"] = df_yagpt["user_prompt"].apply(extract_headline_from_user_prompt)
df_yagpt["lead"] = df_yagpt["user_prompt"].apply(extract_lead_from_user_prompt)

df_mistral["headline"] = df_mistral["user_prompt"].apply(extract_headline_from_user_prompt)
df_mistral["lead"] = df_mistral["user_prompt"].apply(extract_lead_from_user_prompt)

df_gpt4o["headline"] = df_gpt4o["user_prompt"].apply(extract_headline_from_user_prompt)
df_gpt4o["lead"] = df_gpt4o["user_prompt"].apply(extract_lead_from_user_prompt)

In [22]:
dataset = pd.merge(dataset, df_gigachat_lite[["headline", "lead", "gigachat_lite_preds"]], on=["headline", "lead"])
dataset = pd.merge(dataset, df_gigachat_max[["headline", "lead", "gigachat_max_pred"]], on=["headline", "lead"])
dataset = pd.merge(dataset, df_yagpt[["headline", "lead", "yagpt_pred"]], on=["headline", "lead"])
dataset = pd.merge(dataset, df_mistral[["headline", "lead", "mistral_nemo_preds"]], on=["headline", "lead"])
dataset = pd.merge(dataset, df_gpt4o[["gpt4o_explain"]], left_index=True, right_index=True)

In [23]:
print(dataset.shape)

(2500, 12)


In [24]:
dataset.head(1)

,annotations,summary,is_wordplay,date,article_url,headline,lead,gigachat_lite_preds,gigachat_max_pred,yagpt_pred,mistral_nemo_preds,gpt4o_explain
0,None,Жилищный нацпроект помимо количественных целей...,False,2021-12-16,https://www.kommersant.ru/doc/5130217,Комфортная среда,Городскую среду улучшают с учетом мнения жителей,"В заголовке новости ""Комфортная среда"" нет игр...",в заголовке нет игры слов.,в заголовке нет игры слов.,"В заголовке новости ""Комфортная среда"" нет игр...",в заголовке нет игры слов


In [25]:
dataset = dataset.dropna(subset=["annotations"])

In [26]:
dataset.shape

(1241, 12)

In [27]:
EXPLIAN_COLUMNS = ["gigachat_lite_preds", "gigachat_max_pred", "yagpt_pred", "mistral_nemo_preds", "gpt4o_explain"]

for explain_column in EXPLIAN_COLUMNS:
    dataset[f"{explain_column}_search_results"] = dataset.apply(
        lambda row: check_interpretation(
            row["annotations"], row[explain_column]
        ),
        axis=1
    )

In [28]:
for explain_column in EXPLIAN_COLUMNS:
    print(explain_column, dataset[f"{explain_column}_search_results"].isna().sum())

gigachat_lite_preds 207
gigachat_max_pred 207
yagpt_pred 207
mistral_nemo_preds 207
gpt4o_explain 207


In [32]:
res_ = dataset[[f"{t}_search_results" for t in EXPLIAN_COLUMNS]].dropna().sum() / len(dataset.dropna())

In [45]:
pd.DataFrame(res_)[0].astype(float).round(2)

gigachat_lite_preds_search_results    0.21
gigachat_max_pred_search_results      0.30
yagpt_pred_search_results             0.24
mistral_nemo_preds_search_results     0.32
gpt4o_explain_search_results          0.45
Name: 0, dtype: float64